# 05-03 동영상 저장

## 이전 동영상 플레이

### 동영상 재생

In [ ]:
import cv2
import numpy as np

capture = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4')

while True:
    ret, frame = capture.read()
    if not ret: break

    cv2.imshow('video', frame)
    if cv2.waitKey(1) == ord('q'): break

capture.release()
cv2.destroyAllWindows()

### 프레임 처리함수 다시

In [3]:
def process_frame(frame):
    h, w = frame.shape[:2]

    # 그레이 변환 후 3채널로 복원(시각화 통일 위해)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # 좌측 상단에 미니맵(축소 원본) 삽입
    mini = cv2.resize(frame, (w//4, h//4), interpolation=cv2.INTER_AREA)
    gray_bgr[10:10+mini.shape[0], 10:10+mini.shape[1]] = mini

    # 워터마크 텍스트
    cv2.putText(gray_bgr, 'OpenCV Video Processing', (10, h-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    
    return gray_bgr

In [ ]:
import cv2
import numpy as np

inp = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4', cv2.CAP_FFMPEG)
if not inp.isOpened(): inp = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4')
if not inp.isOpened(): raise RuntimeError('입력 영상을 열 수 없습니다.')

fps = inp.get(cv2.CAP_PROP_FPS) or 30     # FPT
w   = int(inp.get(cv2.CAP_PROP_FRAME_WIDTH))   # 해상도 넓이
h   = int(inp.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 해상도 높이
size = (w, h)

# 코덱: mp4 파일이면 보통 'mp4v'가 무난 (플랫폼별 가용 코덱이 다름)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./result/output.mp4', fourcc, fps, size)

while True:
    ret, frame = inp.read()
    if not ret: break

    frame = process_frame(frame)   # 앞 절의 처리 파이프라인
    out.write(frame)               # 프레임 저장

inp.release()
out.release()

print('저장 완료: output.mp4')

저장 완료: output.mp4


# 05-04 전체영상 ROI 크롭 저장

## ROI 영역 저장

In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4', cv2.CAP_FFMPEG)
if not cap.isOpened(): cap = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4',)
if not cap.isOpened(): raise RuntimeError('입력 영상을 열 수 없습니다.')

ret, first = cap.read()
if not ret: raise RuntimeError('첫 프레임을 읽을 수 없습니다.')

# ROI 선택 (드래그) → (x, y, w, h)
x, y, w, h = cv2.selectROI('Pick ROI', first, fromCenter=False, showCrosshair=True)
cv2.destroyWindow('Pick ROI')
if w == 0 or h == 0:
    cap.release()
    raise SystemExit('ROI를 선택하지 않아 종료합니다')

fps = cap.get(cv2.CAP_PROP_FPS) or 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./result/cropped.mp4', fourcc, fps, (int(w), int(h)))

# 첫 프레임부터 다시 읽기
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while True:
    ret, frame = cap.read()
    if not ret: break
    crop = frame[int(y):int(y+h), int(x):int(x+w)]
    out.write(crop)

cap.release()
out.release()
print('저장 완료: cropped.mp4')

저장 완료: cropped.mp4


# 05-05 스냅샷 추출

## 스냅샷 저장

In [10]:
import cv2, os

cap = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4', cv2.CAP_FFMPEG)
if not cap.isOpened(): cap = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4')
if not cap.isOpened(): raise RuntimeError('영상을 열 수 없습니다.')

fps = cap.get(cv2.CAP_PROP_FPS) or 30
interval_sec = 5  # 5초마다
frame_gap = int(fps * interval_sec)

os.makedirs('./thumbs', exist_ok=True)
idx = 0

while True:
    ret, frame = cap.read()
    if not ret: break
    pos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    if pos % frame_gap == 0:
        cv2.imwrite(f'./thumbs/frame_{pos:06d}.jpg', frame)
        idx += 1

cap.release()
print(f'썸네일 저장 수: {idx}')

썸네일 저장 수: 6
